In [126]:
import numpy as np
import pandas as pd

In [127]:
data = pd.read_csv("data.csv")
data.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [128]:
data.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

In [129]:
columns = [
    'Make',
    'Model',
    'Year',
    'Engine HP',
    'Engine Cylinders',
    'Transmission Type',
    'Vehicle Style',
    'highway MPG',
    'city mpg',
    'MSRP'
]
data = data[columns]

In [130]:
df.columns

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg',
       'price', 'above_average'],
      dtype='object')

In [131]:
data.columns = data.columns.str.replace(' ', '_').str.lower()

In [132]:
data.columns

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg',
       'msrp'],
      dtype='object')

In [133]:
data.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
city_mpg              0
msrp                  0
dtype: int64

In [134]:
data = data.fillna(0)

In [135]:
data.isnull().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
city_mpg             0
msrp                 0
dtype: int64

In [136]:
data = data.rename(columns={'msrp': 'price'})

In [137]:
data.columns

Index(['make', 'model', 'year', 'engine_hp', 'engine_cylinders',
       'transmission_type', 'vehicle_style', 'highway_mpg', 'city_mpg',
       'price'],
      dtype='object')

In [138]:
data['transmission_type'].mode()

0    AUTOMATIC
Name: transmission_type, dtype: object

In [139]:
data.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
price                  int64
dtype: object

In [140]:
numericals=["year", "engine_hp", "engine_cylinders", "highway_mpg", "city_mpg", "price"]
numericals

['year', 'engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'price']

In [141]:
val=data[numericals].corr()
val

,year,engine_hp,engine_cylinders,highway_mpg,city_mpg,price
year,1.000000,0.338714,-0.040708,0.258240,0.198171,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.415707,-0.424918,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.614541,-0.587306,0.526274
highway_mpg,0.258240,-0.415707,-0.614541,1.000000,0.886829,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.886829,1.000000,-0.157676
price,0.227590,0.650095,0.526274,-0.160043,-0.157676,1.000000


In [142]:
price_mean = data.price.mean()
data['above_average'] = data.price > price_mean
data.above_average = data.above_average.astype(int)

In [143]:
data['above_average'].head()

0    1
1    1
2    0
3    0
4    0
Name: above_average, dtype: int32

In [144]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import mutual_info_score, accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [150]:
train, test = train_test_split(data, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.25, random_state=42)

# Remove the target variable from the datasets
X_train = train.drop(columns=["above_average"])
y_train = train["above_average"]
X_val = val.drop(columns=["above_average"])
y_val = val["above_average"]
X_test = test.drop(columns=["above_average"])
y_test = test["above_average"]



In [151]:
# Check the shapes of the resulting sets
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)

X_train shape: (7148, 10)
X_val shape: (2383, 10)
X_test shape: (2383, 10)


In [158]:
categorical_columns = ["make", "model", "transmission_type", "vehicle_style"]

mi_scores = {}
for column in categorical_columns:
    mi_score = mutual_info_score(X_train[column], y_train)
    mi_scores[column] = round(mi_score, 2)

# Print the mutual information scores
for column, score in mi_scores.items():
    print(f"Mutual Information between {column} and above_average: {score:.2f}")
    
# Find the variable with the lowest mutual information score
lowest_mi_variable = min(mi_scores, key=mi_scores.get)
lowest_mi_score = mi_scores[lowest_mi_variable]

print(f"The variable with the lowest mutual information score is '{lowest_mi_variable}' with a score of {lowest_mi_score:.2f}")

Mutual Information between make and above_average: 0.24
Mutual Information between model and above_average: 0.46
Mutual Information between transmission_type and above_average: 0.02
Mutual Information between vehicle_style and above_average: 0.08
The variable with the lowest mutual information score is 'transmission_type' with a score of 0.02


In [213]:
# Create a copy of the original data
data_copy = data.copy()

In [214]:
# Select the categorical columns
categorical_columns = ["make", "model", "transmission_type", "vehicle_style"]

In [215]:
# Combine training and validation datasets for one-hot encoding
X_combined = pd.concat([X_train[categorical_columns], X_val[categorical_columns]])


In [216]:
encoder = OneHotEncoder(sparse=False, drop='first')
X_combined_encoded = encoder.fit_transform(X_combined)


C:\Users\koush\anaconda3\envs\ml-zoomcamp\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [217]:
X_train_encoded = X_combined_encoded[:len(X_train)]
X_val_encoded = X_combined_encoded[len(X_train):]


In [218]:
# Create and train the logistic regression model
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train_encoded, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [220]:
# Get the coefficients from the internal estimator
coefficients = model.coef_[0]

In [221]:
# Make predictions on the validation dataset
y_val_pred = model.predict(X_val_encoded)



In [222]:
# Calculate the accuracy on the validation dataset
accuracy = accuracy_score(y_val, y_val_pred)
accuracy



0.9332773814519513

In [232]:
#Round the accuracy to 2 decimal places
rounded_accuracy = round(accuracy, 2)

print(f"Validation Accuracy: {rounded_accuracy:.2f}")

Validation Accuracy: 0.93


In [233]:
# Get the coefficients of the logistic regression model
coefficients = model.coef_[0]

In [235]:
# Find the index of the least useful feature (the one with the smallest absolute coefficient)
least_useful_feature_index = np.argmin(np.abs(coefficients))

In [237]:
print(f"The least useful feature is feature number {least_useful_feature_index} with a coefficient of {coefficients[least_useful_feature_index]:.4f}")

The least useful feature is feature number 99 with a coefficient of 0.0000


In [247]:
# Create a function to calculate accuracy without a given feature
def calculate_accuracy_without_feature(feature_index):
    X_train_encoded_without_feature = np.delete(X_train_encoded, feature_index, axis=1)
    X_val_encoded_without_feature = np.delete(X_val_encoded, feature_index, axis=1)

    model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model.fit(X_train_encoded_without_feature, y_train)

    y_val_pred = model.predict(X_val_encoded_without_feature)

    accuracy = accuracy_score(y_val, y_val_pred)
    return accuracy

In [248]:
# Calculate the original accuracy
original_accuracy = accuracy

# Initialize a list to store feature accuracy differences
feature_accuracy_differences = []

In [250]:
# Iterate through each feature
for feature_index in range(X_train_encoded.shape[1]):
    
    # Calculate accuracy without the current feature
    accuracy_without_feature = calculate_accuracy_without_feature(feature_index)
    
    # Calculate the accuracy difference
    accuracy_difference = original_accuracy - accuracy_without_feature
    
    # Store the feature index and its accuracy difference in the list
    feature_accuracy_differences.append((feature_index, accuracy_difference))



In [252]:

# Sort the features by their accuracy differences (from least useful to most useful)
sorted_feature_accuracy_differences = sorted(feature_accuracy_differences, key=lambda x: x[1])

In [253]:
# Find and print the least useful feature
least_useful_feature, accuracy_difference = sorted_feature_accuracy_differences[0]

print(f"The least useful feature is '{least_useful_feature}' with an accuracy difference of {accuracy_difference:.4f}")

The least useful feature is '315' with an accuracy difference of -0.0017


In [257]:
# Create a function to calculate accuracy without a given feature
def calculate_accuracy_without_feature(feature_index):
    X_train_encoded_without_feature = np.delete(X_train_encoded, feature_index, axis=1)
    X_val_encoded_without_feature = np.delete(X_val_encoded, feature_index, axis=1)

    model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    model.fit(X_train_encoded_without_feature, y_train)

    y_val_pred = model.predict(X_val_encoded_without_feature)

    accuracy = accuracy_score(y_val, y_val_pred)
    return accuracy

# Calculate the original accuracy
original_accuracy = accuracy

# Initialize a list to store feature accuracy differences
feature_accuracy_differences = []

# Iterate through each feature
for feature_index in range(X_train_encoded.shape[1]):
    
    # Calculate accuracy without the current feature
    accuracy_without_feature = calculate_accuracy_without_feature(feature_index)
    
    # Calculate the accuracy difference
    accuracy_difference = original_accuracy - accuracy_without_feature
    
    # Append the accuracy difference to the list
    feature_accuracy_differences.append(accuracy_difference)

# Find the least useful feature's index
least_useful_feature_index = np.argmin(feature_accuracy_differences)

# Print the least useful feature's index and its accuracy difference
print(f"The least useful feature (by index) is feature number {least_useful_feature_index} with an accuracy difference of {feature_accuracy_differences[least_useful_feature_index]:.4f}")


The least useful feature (by index) is feature number 315 with an accuracy difference of -0.0017


In [259]:
# Define the list of feature indices
feature_indices = [2, 3, 4, 7]  # 'year,' 'engine_hp,' 'transmission_type,' 'city_mpg'

# Initialize a list to store feature accuracy differences
feature_accuracy_differences = []

# Iterate through each feature index
for feature_index in feature_indices:
    
    # Calculate accuracy without the current feature
    accuracy_without_feature = calculate_accuracy_without_feature(feature_index)
    
    # Calculate the accuracy difference
    accuracy_difference = original_accuracy - accuracy_without_feature
    
    # Append the accuracy difference to the list
    feature_accuracy_differences.append(accuracy_difference)

# Find the feature index with the smallest accuracy difference
smallest_difference_index = np.argmin(feature_accuracy_differences)
smallest_accuracy_difference = feature_accuracy_differences[smallest_difference_index]

# Map feature indices to their names
feature_names = ['year', 'engine_hp', 'transmission_type', 'city_mpg']
smallest_difference_feature = feature_names[smallest_difference_index]

print(f"The feature with the smallest accuracy difference is '{smallest_difference_feature}' with a difference of {smallest_accuracy_difference:.4f}")


The feature with the smallest accuracy difference is 'engine_hp' with a difference of -0.0008


In [260]:
from math import sqrt
from sklearn.linear_model import Ridge

In [261]:
# Apply logarithmic transformation to the 'price' column
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

In [262]:
# List of alpha values to try
alpha_values = [0, 0.01, 0.1, 1, 10]

In [263]:
# Initialize a dictionary to store RMSE scores for different alphas
rmse_scores = {}

# Iterate through each alpha value
for alpha in alpha_values:
    # Create and fit the Ridge regression model with 'sag' solver
    model = Ridge(alpha=alpha, solver='sag', random_state=42)
    model.fit(X_train_encoded, y_train_log)

    # Make predictions on the validation set
    y_val_pred_log = model.predict(X_val_encoded)

    # Convert predicted values back to their original scale using inverse transformation
    y_val_pred = np.expm1(y_val_pred_log)

    # Calculate RMSE (Root Mean Squared Error)
    rmse = sqrt(mean_squared_error(y_val, y_val_pred))

    # Round RMSE to 3 decimal digits and store in the dictionary
    rmse_scores[alpha] = round(rmse, 3)

In [264]:
# Print RMSE scores for different alpha values
for alpha, rmse in rmse_scores.items():
    print(f"Alpha = {alpha}: RMSE = {rmse:.3f}")

Alpha = 0: RMSE = 0.221
Alpha = 0.01: RMSE = 0.221
Alpha = 0.1: RMSE = 0.221
Alpha = 1: RMSE = 0.225
Alpha = 10: RMSE = 0.265
